## Яндекс Практикум, курс "Инженер Машинного Обучения" (2024 г.)
## Проект 4-го спринта: "Создание рекомендательной системы"

## Этап 4. Сервис рекомендаций

__Постановка задачи__

Напишите FastAPI-микросервис для выдачи рекомендаций, который:
- принимает запрос с идентификатором пользователя и выдаёт рекомендации,
- учитывает историю пользователя,
- смешивает онлайн- и оффлайн-рекомендации.

В README опишите стратегию смешивания онлайн- и оффлайн-рекомендаций.

Выполните тестирование микросервиса в ноутбуке `part_3_test.ipynb`:
- для пользователя без персональных рекомендаций,
- для пользователя с персональными рекомендациями, но без онлайн-истории,
- для пользователя с персональными рекомендациями и онлайн-историей.

### Инициализация

In [1]:
import numpy as np
import pandas as pd
import requests

Задаем url-адреса трех веб-сервисов

In [2]:
# Основной сервис для получения оффлайн- и онлайн-рекомендаций
recommendations_url = "http://127.0.0.1:8000"

# Вспомогательный сервис для получения рекомендаций по умолчанию на основе топ-треков
features_store_url = "http://127.0.0.1:8010"

# Вспомогательный сервис для хранения и получения последних онлайн-событий пользователя
events_store_url = "http://127.0.0.1:8020"

In [3]:
# Общий заголовок для всех http-запросов
headers = {"Content-type": "application/json", "Accept": "text/plain"}

Чтобы запустить тестируемые сервисы, нужно выполнить 3 команды (по одному на каждый сервис) в 3-х разных терминалах, находясь в папке проекта:
```
uvicorn recommendations_service:app
uvicorn events_service:app --port 8020
uvicorn features_service:app --port 8010
```

### Загрузка данных

Загрузким треки для отображения рекомендаций со всеми полями, а также сами рекомендации, чтобы выбирать из них пользователей и объекты для тестирования

In [5]:
items = pd.read_parquet("items.parquet")
recommendations = pd.read_parquet('recommendations.parquet')
similar = pd.read_parquet('similar.parquet')
top_popular = pd.read_parquet('top_popular.parquet')

### Получение рекомендаций по умолчанию

В этом сценарии мы получаем рекомендации на основе топ-треков, когда у пользователя нет истории и соответственно нет персональных оффлайн-рекомендаций.

Напишем вспомогательную функцию для отображения объектов по их идентификаторам

In [6]:
def display_items(item_ids):

    item_columns_to_use = ["item_id", "name", "genres", "albums", "artists"]
    
    items_selected = items.query("item_id in @item_ids")[item_columns_to_use]
    items_selected = items_selected.set_index("item_id").reindex(item_ids)
    items_selected = items_selected.reset_index()
    
    display(items_selected)

Посмотрим топ-рекомендации из соответствующего файла

In [7]:
top_popular.head(10)

,rank,item_id,plays,users,name,genres,artists,albums
0,1,53404,15704,15704,Smells Like Teen Spirit,"[alternative, allrock, rock]",[Nirvana],[Smells Like Teen Spirit / In Bloom / On A Pla...
1,2,178529,14088,14088,Numb,"[numetal, metal]",[Linkin Park],"[Meteora, 00s Rock Anthems]"
2,3,37384,12798,12798,Zombie,"[allrock, rock]",[The Cranberries],"[90s Alternative, MNM Sing Your Song: Back To ..."
3,4,6705392,12735,12735,Seven Nation Army,[alternative],[The White Stripes],"[Pay Close Attention : XL Recordings, Radio 1 ..."
4,5,33311009,11972,11972,Believer,"[allrock, rock]",[Imagine Dragons],"[Horoscope Tunes: Cancer, Workout Music Hits 2..."
5,6,328683,11885,11885,Bring Me To Life,"[alternative, allrock, rock]",[Evanescence],"[Mental Health 2020, Monster Millennium Hits, ..."
6,7,148345,11802,11802,Californication,"[allrock, rock]",[Red Hot Chili Peppers],"[The Studio Album Collection 1991-2011, Greate..."
7,8,795836,11731,11731,Shape Of My Heart,"[allrock, pop, rock]",[Sting],"[25 Years, Сентиментальные рок-песни, Ten Summ..."
8,9,630670,11722,11722,"You're Gonna Go Far, Kid",[punk],[The Offspring],"[You're Gonna Go Far, Kid, Sleep Music Rock Ed..."
9,10,178495,11522,11522,In the End,"[numetal, metal]",[Linkin Park],"[Antyradio: Najlepszy Rock Na Swiecie Vol. 4, ..."


Отправим запрос для получения рекомендаций по умолчанию и сравним его резульат с данными выше

In [8]:
params = {'k': 10}
resp = requests.post(recommendations_url + "/recommendations_default", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,53404,Smells Like Teen Spirit,"[alternative, allrock, rock]",[Smells Like Teen Spirit / In Bloom / On A Pla...,[Nirvana]
1,178529,Numb,"[numetal, metal]","[Meteora, 00s Rock Anthems]",[Linkin Park]
2,37384,Zombie,"[allrock, rock]","[90s Alternative, MNM Sing Your Song: Back To ...",[The Cranberries]
3,6705392,Seven Nation Army,[alternative],"[Pay Close Attention : XL Recordings, Radio 1 ...",[The White Stripes]
4,33311009,Believer,"[allrock, rock]","[Horoscope Tunes: Cancer, Workout Music Hits 2...",[Imagine Dragons]
5,328683,Bring Me To Life,"[alternative, allrock, rock]","[Mental Health 2020, Monster Millennium Hits, ...",[Evanescence]
6,148345,Californication,"[allrock, rock]","[The Studio Album Collection 1991-2011, Greate...",[Red Hot Chili Peppers]
7,795836,Shape Of My Heart,"[allrock, pop, rock]","[25 Years, Сентиментальные рок-песни, Ten Summ...",[Sting]
8,630670,"You're Gonna Go Far, Kid",[punk],"[You're Gonna Go Far, Kid, Sleep Music Rock Ed...",[The Offspring]
9,178495,In the End,"[numetal, metal]","[Antyradio: Najlepszy Rock Na Swiecie Vol. 4, ...",[Linkin Park]


Результаты запроса и данные в файле совпадают.

### Получение персональных рекомендаций без онлайн-истории

В этом сценарии рекомендации извлекаются по идентификтору пользователя из данных, сгенерированных на основании истории пользователя.

In [9]:
# Выберем случайного пользователя из оффлайн-рекомендаций
user_id = recommendations['user_id'].sample().iat[0]
print(f"user_id выбранного пользователя: {user_id}")

user_id выбранного пользователя: 288720


In [12]:
# Посмотрим, какие для него есть рекомендации
recommendations.query('user_id == @user_id').head(10)

,user_id,item_id,tracks_played_by_user,genre_0,genre_20,genre_2,genre_11,genre_6,genre_74,genre_21,genre_4,genre_14,genre_10,genre_others,score,rank
703900,288720,78608850,954,0.121681,0.221523,0.00891,0.090671,0.001572,0.144916,0.067173,0.001572,0.069182,0.006639,0.26616,0.740319,1
703901,288720,84019953,954,0.121681,0.221523,0.00891,0.090671,0.001572,0.144916,0.067173,0.001572,0.069182,0.006639,0.26616,0.721985,2
703903,288720,74611690,954,0.121681,0.221523,0.00891,0.090671,0.001572,0.144916,0.067173,0.001572,0.069182,0.006639,0.26616,0.676690,3
703904,288720,81002414,954,0.121681,0.221523,0.00891,0.090671,0.001572,0.144916,0.067173,0.001572,0.069182,0.006639,0.26616,0.672103,4
703905,288720,80153541,954,0.121681,0.221523,0.00891,0.090671,0.001572,0.144916,0.067173,0.001572,0.069182,0.006639,0.26616,0.668791,5
703902,288720,77999637,954,0.121681,0.221523,0.00891,0.090671,0.001572,0.144916,0.067173,0.001572,0.069182,0.006639,0.26616,0.659239,6
703906,288720,52395280,954,0.121681,0.221523,0.00891,0.090671,0.001572,0.144916,0.067173,0.001572,0.069182,0.006639,0.26616,0.652553,7
703907,288720,48592141,954,0.121681,0.221523,0.00891,0.090671,0.001572,0.144916,0.067173,0.001572,0.069182,0.006639,0.26616,0.631554,8
703908,288720,69374769,954,0.121681,0.221523,0.00891,0.090671,0.001572,0.144916,0.067173,0.001572,0.069182,0.006639,0.26616,0.616929,9
703909,288720,65320578,954,0.121681,0.221523,0.00891,0.090671,0.001572,0.144916,0.067173,0.001572,0.069182,0.006639,0.26616,0.608228,10


Выполним запрос и сравним его результат с данными выше

In [11]:
# Отправляем запрос для получения только оффлайн-рекомендаций
params = {'user_id': user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    recs = []
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,78608850,Нету интереса,"[rusrap, rap]",[Нету интереса],[10AGE]
1,84019953,Close Eyes,"[rap, foreignrap]",[Close Eyes],[DVRST]
2,74611690,Чистый,"[rusrap, rap]",[Чистый],[Скриптонит]
3,81002414,ночь,"[pop, ruspop]","[ночь, мои (твои) тёмные желания]",[ooes]
4,80153541,Патрон,"[rusrap, rap]",[Патрон],[Miyagi & Andy Panda]
5,77999637,Silhouette Challenge,"[rap, foreignrap]",[Silhouette Challenge],[DJB]
6,52395280,Paris,[electronics],[Paris],[Else]
7,48592141,Положение,"[rusrap, rap]",[Уроборос: Улица 36],[Скриптонит]
8,69374769,Deep End,[rnb],"[time machine, Deep End]",[Fousheé]
9,65320578,Brooklyn,"[rusrap, rap]",[Brooklyn],"[Miyagi & Andy Panda, TumaniYO]"


По `item_id` видим, что результаты запроса и данные в файле совпадают.

### Получение персональных рекомендаций с онлайн-историей

В этом сценарии к оффлайн-рекомендациям из предыдущего раздела добавляются объекты, похожие на 3 последних прослушанных трека из онлайн-истории.

Возьмем 3 произвольных трека из i2i-рекомендаций и добавим их в онлайн-историю того же пользователя, что и в предыдущем тестовом сценарии

In [13]:
# Выбираем 3 объекта из i2i-рекомендаций
item_ids = similar['item_id_1'].unique()[:3]
print(item_ids)

# Добавляем их в онлайн-историю пользователя
for item_id in item_ids:
    params = {"user_id": user_id, "item_id": item_id}
    resp = requests.post(events_store_url + "/put", headers=headers, params=params)
    if resp.status_code == 200:
        resp = resp.json()
        print(resp['result'])    
    else:
        print(f"status code: {resp.status_code}") 

[549 553 558]
ok
ok
ok


Проверим, что события добавились, причем в обратном порядке

In [14]:
params = {"user_id": user_id, "k": 3}
resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    print(resp['events'])    
else:
    print(f"status code: {resp.status_code}")

[558, 553, 549]


Протестируем работу сервиса `feautures_service`. 

Возмьмем последний объект из списка выше и найдем похожие на него треки из файла `similar.parquet`

In [15]:
item_id_1 = item_ids[-1]
similar.query('item_id_1 == @item_id_1').head(10)

,score,item_id_1,item_id_2
23,0.924216,558,1706463
24,0.908504,558,1706462
25,0.904736,558,550048
26,0.897181,558,69658
27,0.894441,558,553
28,0.886535,558,71239
29,0.885370,558,2277920
30,0.881694,558,71965
31,0.881250,558,34300
32,0.873259,558,10260869


Теперь получим тот же набор объектов с помощью сервиса `features_service`

In [16]:
params = {"item_id": item_id_1, "k": 10}
resp = requests.post(features_store_url +"/similar_items", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    print(resp['item_id_2'])    
else:
    print(f"status code: {resp.status_code}")    

[1706463, 1706462, 550048, 69658, 553, 71239, 2277920, 71965, 34300, 10260869]


Видим, что данные совпадают.

Протестируем запрос к сервису `reccomendations_service` для получения объектов, похожих на 3 последних трека из онлайн-истории выбранного пользователя

In [17]:
params = {'user_id': user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    recs = []
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,1706462,The Time (Dirty Bit),"[electronics, pop]","[Kids Songs: Game Day!, Party de Revelion 2021...",[Black Eyed Peas]
1,1706463,Don’t Stop The Party,"[electronics, pop]","[The Beginning, NRJ 200% Hits 2011 Vol 2, The ...",[Black Eyed Peas]
2,26771,London Bridge,[rnb],"[Faxina Pop, Летняя дискотека, Ready Steady Ib...",[Fergie]
3,4198,Fergalicious,[rnb],"[Chicks With Kick, The Dutchess Deluxe, Pregam...","[will.i.am, Fergie]"
4,69658,I Got It From My Mama,[electronics],"[Mom Songs, Hip Hop: The Collection 2008, Ulti...",[will.i.am]
5,37402640,Impacto,"[latinfolk, folk]",[El Cartel: The Big Boss],"[Daddy Yankee, Fergie]"
6,631538,Boom Boom Pow,[rnb],"[Now 20, Boom Boom Pow]",[Black Eyed Peas]
7,123846,Lose Control,"[rap, foreignrap]","[Respect M.E., The Cookbook]","[Ciara, Fatman Scoop, Missy Elliott]"
8,12281,My Humps,"[rap, foreignrap]","[Monkey Business, Junior Prom, Horoscope Tunes...",[Black Eyed Peas]
9,78710,Party People,"[rap, rnb, foreignrap]","[The Dutchess Deluxe EP, Brass Knuckles, The D...","[Nelly, Fergie]"


Видим, что результат содержит в т.ч. объекты, похожие на последий трек из онлайн-истории пользователя.

Наконец, получим смешанные рекомендации. На нечетных местах будут оффлайн-рекомендации, на четных - онлайн.

In [18]:
params = {"user_id": user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,78608850,Нету интереса,"[rusrap, rap]",[Нету интереса],[10AGE]
1,1706462,The Time (Dirty Bit),"[electronics, pop]","[Kids Songs: Game Day!, Party de Revelion 2021...",[Black Eyed Peas]
2,84019953,Close Eyes,"[rap, foreignrap]",[Close Eyes],[DVRST]
3,1706463,Don’t Stop The Party,"[electronics, pop]","[The Beginning, NRJ 200% Hits 2011 Vol 2, The ...",[Black Eyed Peas]
4,74611690,Чистый,"[rusrap, rap]",[Чистый],[Скриптонит]
5,26771,London Bridge,[rnb],"[Faxina Pop, Летняя дискотека, Ready Steady Ib...",[Fergie]
6,81002414,ночь,"[pop, ruspop]","[ночь, мои (твои) тёмные желания]",[ooes]
7,4198,Fergalicious,[rnb],"[Chicks With Kick, The Dutchess Deluxe, Pregam...","[will.i.am, Fergie]"
8,80153541,Патрон,"[rusrap, rap]",[Патрон],[Miyagi & Andy Panda]
9,69658,I Got It From My Mama,[electronics],"[Mom Songs, Hip Hop: The Collection 2008, Ulti...",[will.i.am]
